# IMPORTS

In [1]:
import pandas as pd
import no_transfer_linear, no_transfer_lstm
import global_linear_linear, global_linear_lstm, global_lstm_linear, global_lstm_lstm
import torch
import utils
import pickle
import numpy as np
import random


In [2]:
IS_CUDA = False
IS_SHORT = True

device_config = "cude" if IS_CUDA else "cpu"
train_steps = 1 if IS_SHORT else 1000
modes = [
    "no_transfer_linear",
    "no_transfer_lstm",
    "global_linear_linear",
    "global_lstm_linear",
    "global_linear_lstm",
    "global_lstm_lstm",
]
MODE = modes[5]
MODE

'global_lstm_lstm'

# LOOP

In [ ]:
# data params
manualSeed = 999999999
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

for MODE in modes:
    data_config = {
        "data_path": ".\\Tasks\\",
        "region": ["Americas", "Europe", "Asia and Pacific", "MEA"],
        "Europe": [
            "Europe_AEX",
            "Europe_ASE",
            "Europe_ATX",
            "Europe_BEL20",
            "Europe_BUX",
            "Europe_BVLX",
            "Europe_CAC",
            "Europe_CYSMMAPA",
            "Europe_DAX",
            "Europe_HEX",
            "Europe_IBEX",
            "Europe_ISEQ",
            "Europe_KFX",
            "Europe_OBX",
            "Europe_OMX",
            "Europe_SMI",
            "Europe_UKX",
            "Europe_VILSE",
            "Europe_WIG20",
            "Europe_XU100",
            "Europe_SOFIX",
            "Europe_SBITOP",
            "Europe_PX",
            "Europe_CRO",
        ],
        "Asia and Pacific": [
            "Asia and Pacific_AS51",
            "Asia and Pacific_FBMKLCI",
            "Asia and Pacific_HSI",
            "Asia and Pacific_JCI",
            "Asia and Pacific_KOSPI",
            "Asia and Pacific_KSE100",
            "Asia and Pacific_NIFTY",
            "Asia and Pacific_NKY",
            "Asia and Pacific_NZSE50FG",
            "Asia and Pacific_PCOMP",
            "Asia and Pacific_STI",
            "Asia and Pacific_SHSZ300",
            "Asia and Pacific_TWSE",
        ],
        "Americas": [
            "Americas_IBOV",
            "Americas_MEXBOL",
            "Americas_MERVAL",
            "Americas_SPTSX",
            "Americas_SPX",
            "Americas_RTY",
        ],
        "MEA": [
            "MEA_DFMGI",
            "MEA_DSM",
            "MEA_EGX30",
            "MEA_FTN098",
            "MEA_JOSMGNFF",
            "MEA_KNSMIDX",
            "MEA_KWSEPM",
            "MEA_MOSENEW",
            "MEA_MSM30",
            "MEA_NGSE30",
            "MEA_PASISI",
            "MEA_SASEIDX",
            "MEA_SEMDEX",
            "MEA_TA-35",
            "MEA_TOP40",
        ],
        "additional_data_path": "_all_assets_data.pkl.gz",
    }

    # problem params
    problem_config = {
        "export_path": ".\\Results_model\\",
        # "export_path": ".\\",
        "val_period": 0,  # if val is 0, then its results are the same as training
        "holdout_period": 252 * 3,
    }

    # model params
    model_config = {
        "train_steps": train_steps,
        "tasks_train_steps": 0,  # [len(data_config[x]) for x in data_config["region"]] - right
        "batch_size": 32,
        "seq_len": 252,
        "transfer_strat": MODE,
        "device": torch.device(device_config),
        "export_losses": False,
        "no_transfer_linear": {
            "optimizer_learning_rate": 0.001,
            "amsgrad": True,
            "export_weights": False,
        },
        "no_transfer_lstm": {
            "optimizer_learning_rate": 0.001,
            "amsgrad": True,
            "export_model": False,
            "out_n_hi": 50,
            "n_layers": 2,
            "drop_rate": 0.1,
        },
        "global_linear_linear": {
            "optimizer_learning_rate": 0.01,
            "amsgrad": True,
            "export_weights": False,
            "in_transfer_dim": 200,
            "out_transfer_dim": 200,
        },
        "global_lstm_linear": {
            "optimizer_learning_rate": 0.01,
            "amsgrad": True,
            "export_model": False,
            "in_transfer_dim": 5,
            "out_transfer_dim": 5,
            "n_layers": 2,
            "drop_rate": 0.1,
        },
        "global_linear_lstm": {
            "optimizer_learning_rate": 0.01,
            "amsgrad": True,
            "export_model": False,
            "in_transfer_dim": 5,
            "out_transfer_dim": 5,
            "in_n_layers": 2,
            "out_n_layers": 2,
            "out_n_hi": 10,
            "drop_rate": 0.1,
        },
        "global_lstm_lstm": {
            "optimizer_learning_rate": 0.01,
            "amsgrad": True,
            "export_model": False,
            "in_transfer_dim": 5,
            "out_transfer_dim": 5,
            "in_n_layers": 2,
            "out_n_layers": 2,
            "n_layers": 2,
            "out_n_hi": 10,
            "drop_rate": 0.1,
            "drop_rate_transfer": 0.1,
        },
    }

    ## MAIN ROUTINE
    # main routine

    # pre-allocation
    export_label = (
        "valperiod_"
        + str(problem_config["val_period"])
        + "_testperiod_"
        + str(problem_config["holdout_period"])
        + "_tsteps_"
        + str(model_config["train_steps"])
        + "_tksteps_"
        + str(model_config["tasks_train_steps"])
        + "_batchsize_"
        + str(model_config["batch_size"])
        + "_seqlen_"
        + str(model_config["seq_len"])
        + "_transferstrat_"
        + model_config["transfer_strat"]
        + "_lr_"
        + str(model_config[model_config["transfer_strat"]]["optimizer_learning_rate"])
    )
    data_config["export_label"] = export_label
    problem_config["export_label"] = export_label
    model_config["export_label"] = export_label
    model_config["export_path"] = problem_config["export_path"]

    # get data
    Xtrain_tasks, Xval_tasks, Xtest_tasks = utils.get_data(
        data_config, problem_config, model_config
    )

    print(model_config["transfer_strat"])
    # set model
    if model_config["transfer_strat"] == "no_transfer_linear":
        transfer_trad_strat = no_transfer_linear.NoTransferLinear(
            Xtrain_tasks, model_config
        )
        add_label = [""] * len(data_config["region"])

    elif model_config["transfer_strat"] == "no_transfer_lstm":
        transfer_trad_strat = no_transfer_lstm.NoTransferLSTM(
            Xtrain_tasks, model_config
        )
        add_label = [
            "_n_hi_"
            + str(model_config["no_transfer_lstm"]["out_n_hi"])
            + "_n_layers_"
            + str(model_config["no_transfer_lstm"]["n_layers"])
            + "dpr"
            + str(model_config["no_transfer_lstm"]["drop_rate"])
            for x in data_config["region"]
        ]

    elif model_config["transfer_strat"] == "global_linear_linear":
        transfer_trad_strat = global_linear_linear.GlobalLinearLinear(
            Xtrain_tasks, model_config
        )
        add_label = [
            "_indim_"
            + str(model_config["global_linear_linear"]["in_transfer_dim"])
            + "_outdim_"
            + str(model_config["global_linear_linear"]["out_transfer_dim"])
            for x in data_config["region"]
        ]

    elif model_config["transfer_strat"] == "global_lstm_linear":
        transfer_trad_strat = global_lstm_linear.GlobalLSTMLinear(
            Xtrain_tasks, model_config
        )
        add_label = [
            "_indim_"
            + str(model_config["global_lstm_linear"]["in_transfer_dim"])
            + "_outdim_"
            + str(model_config["global_lstm_linear"]["out_transfer_dim"])
            + "_inlay_"
            + str(model_config["global_lstm_linear"]["n_layers"])
            + "dpr"
            + str(model_config["global_lstm_linear"]["drop_rate"])
            for x in data_config["region"]
        ]

    elif model_config["transfer_strat"] == "global_linear_lstm":
        transfer_trad_strat = global_linear_lstm.GlobalLinearLSTM(
            Xtrain_tasks, model_config
        )
        add_label = [
            "_indim_"
            + str(model_config["global_linear_lstm"]["in_transfer_dim"])
            + "_outdim_"
            + str(model_config["global_linear_lstm"]["out_transfer_dim"])
            + "_inlay_"
            + str(model_config["global_linear_lstm"]["in_n_layers"])
            + "_outlay_"
            + str(model_config["global_linear_lstm"]["out_n_layers"])
            + "_lindim_"
            + str(model_config["global_linear_lstm"]["out_n_hi"])
            + "dpr"
            + str(model_config["global_linear_lstm"]["drop_rate"])
            for x in data_config["region"]
        ]

    elif model_config["transfer_strat"] == "global_lstm_lstm":
        transfer_trad_strat = global_lstm_lstm.GlobalLSTMLSTM(
            Xtrain_tasks, model_config
        )
        add_label = [
            "_indim_"
            + str(model_config["global_lstm_lstm"]["in_transfer_dim"])
            + "_outdim_"
            + str(model_config["global_lstm_lstm"]["out_transfer_dim"])
            + "_inlay_"
            + str(model_config["global_lstm_lstm"]["in_n_layers"])
            + "_outlay_"
            + str(model_config["global_lstm_lstm"]["out_n_layers"])
            + "_odim_"
            + str(model_config["global_lstm_lstm"]["out_n_hi"])
            + "_ltr_"
            + str(model_config["global_lstm_lstm"]["n_layers"])
            + "_dpr_"
            + str(model_config["global_lstm_lstm"]["drop_rate"])
            + "_dtr_"
            + str(model_config["global_lstm_lstm"]["drop_rate_transfer"])
            for x in data_config["region"]
        ]

    # additional labelling
    to_add_label = {}
    for lab, region in zip(add_label, data_config["region"]):
        to_add_label[region] = lab

    ## TRAIN
    # train model
    import time

    start = time.time()
    transfer_trad_strat.train()
    print(time.time() - start)

    # get signals
    Xtrain_signal = transfer_trad_strat.predict(Xtrain_tasks)
    Xval_signal = transfer_trad_strat.predict(Xval_tasks)
    Xtest_signal = transfer_trad_strat.predict(Xtest_tasks)

    # compute results
    k = True
    for region in data_config["region"]:
        region_task_paths = [t + "_all_assets_data.pkl.gz" for t in data_config[region]]

        z = True
        for task, tk_path in zip(data_config[region], region_task_paths):
            # get signal
            pred_train = Xtrain_signal[region][task].cpu()
            pred_val = Xval_signal[region][task].cpu()
            pred_test = Xtest_signal[region][task].cpu()

            # get target
            Ytrain = (
                Xtrain_tasks[region][task]
                .view(1, -1, Xtrain_tasks[region][task].size(1))[:, 1:]
                .cpu()
            )
            Yval = (
                Xval_tasks[region][task]
                .view(1, -1, Xval_tasks[region][task].size(1))[:, 1:]
                .cpu()
            )
            Ytest = (
                Xtest_tasks[region][task]
                .view(1, -1, Xtest_tasks[region][task].size(1))[:, 1:]
                .cpu()
            )

            # compute returns
            df_train_ret = (
                pred_train.mul(Ytrain)[0].cpu().numpy()
                - utils.compute_trading_costs(pred_train)[0].cpu().numpy()
            )
            df_val_ret = (
                pred_val.mul(Yval)[0].cpu().numpy()
                - utils.compute_trading_costs(pred_val)[0].cpu().numpy()
            )
            df_test_ret = (
                pred_test.mul(Ytest)[0].cpu().numpy()
                - utils.compute_trading_costs(pred_test)[0].cpu().numpy()
            )

            # get performance metrics
            df = pd.read_pickle(data_config["data_path"] + tk_path)
            df_train_ret = pd.DataFrame(df_train_ret, columns=df.columns)
            df_train_metrics = utils.compute_performance_metrics(df_train_ret)
            df_train_metrics["exchange"] = task

            df_val_ret = pd.DataFrame(df_val_ret, columns=df.columns)
            df_val_metrics = utils.compute_performance_metrics(df_val_ret)
            df_val_metrics["exchange"] = task

            df_test_ret = pd.DataFrame(df_test_ret, columns=df.columns)
            df_test_metrics = utils.compute_performance_metrics(df_test_ret)
            df_test_metrics["exchange"] = task

            if z:
                all_df_train_metrics = df_train_metrics.copy()
                all_df_val_metrics = df_val_metrics.copy()
                all_df_test_metrics = df_test_metrics.copy()
                z = False
            else:
                all_df_train_metrics = pd.concat(
                    [all_df_train_metrics, df_train_metrics], axis=0
                )
                all_df_val_metrics = pd.concat(
                    [all_df_val_metrics, df_val_metrics], axis=0
                )
                all_df_test_metrics = pd.concat(
                    [all_df_test_metrics, df_test_metrics], axis=0
                )

        # export results
        all_df_train_metrics["region"] = region
        all_df_train_metrics["set"] = "train"
        all_df_val_metrics["region"] = region
        all_df_val_metrics["set"] = "val"
        all_df_test_metrics["region"] = region
        all_df_test_metrics["set"] = "test"

        pd.concat(
            [all_df_train_metrics, all_df_val_metrics, all_df_test_metrics], axis=0
        ).to_csv(
            problem_config["export_path"]
            + region
            + "_"
            + problem_config["export_label"]
            + to_add_label[region]
            + ".csv"
        )
        pickle.dump(
            model_config,
            open(
                problem_config["export_path"]
                + region
                + "_"
                + problem_config["export_label"]
                + to_add_label[region]
                + "_modelconfig.pkl.gz",
                "wb",
            ),
        )
        if model_config["export_losses"]:
            pickle.dump(
                transfer_trad_strat.losses[region],
                open(
                    problem_config["export_path"]
                    + region
                    + "_"
                    + problem_config["export_label"]
                    + to_add_label[region]
                    + "_losses.pkl.gz",
                    "wb",
                ),
            )


Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

...

d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

# SEPARATE

In [3]:
# data params
manualSeed = 999999999
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

data_config = {
    "data_path": ".\\Tasks\\",
    "region": ["Americas", "Europe", "Asia and Pacific", "MEA"],
    "Europe": [
        "Europe_AEX",
        "Europe_ASE",
        "Europe_ATX",
        "Europe_BEL20",
        "Europe_BUX",
        "Europe_BVLX",
        "Europe_CAC",
        "Europe_CYSMMAPA",
        "Europe_DAX",
        "Europe_HEX",
        "Europe_IBEX",
        "Europe_ISEQ",
        "Europe_KFX",
        "Europe_OBX",
        "Europe_OMX",
        "Europe_SMI",
        "Europe_UKX",
        "Europe_VILSE",
        "Europe_WIG20",
        "Europe_XU100",
        "Europe_SOFIX",
        "Europe_SBITOP",
        "Europe_PX",
        "Europe_CRO",
    ],
    "Asia and Pacific": [
        "Asia and Pacific_AS51",
        "Asia and Pacific_FBMKLCI",
        "Asia and Pacific_HSI",
        "Asia and Pacific_JCI",
        "Asia and Pacific_KOSPI",
        "Asia and Pacific_KSE100",
        "Asia and Pacific_NIFTY",
        "Asia and Pacific_NKY",
        "Asia and Pacific_NZSE50FG",
        "Asia and Pacific_PCOMP",
        "Asia and Pacific_STI",
        "Asia and Pacific_SHSZ300",
        "Asia and Pacific_TWSE",
    ],
    "Americas": [
        "Americas_IBOV",
        "Americas_MEXBOL",
        "Americas_MERVAL",
        "Americas_SPTSX",
        "Americas_SPX",
        "Americas_RTY",
    ],
    "MEA": [
        "MEA_DFMGI",
        "MEA_DSM",
        "MEA_EGX30",
        "MEA_FTN098",
        "MEA_JOSMGNFF",
        "MEA_KNSMIDX",
        "MEA_KWSEPM",
        "MEA_MOSENEW",
        "MEA_MSM30",
        "MEA_NGSE30",
        "MEA_PASISI",
        "MEA_SASEIDX",
        "MEA_SEMDEX",
        "MEA_TA-35",
        "MEA_TOP40",
    ],
    "additional_data_path": "_all_assets_data.pkl.gz",
}

# problem params
problem_config = {
    "export_path": ".\\Results_model_test\\",
    # "export_path": ".\\",
    "val_period": 0,  # if val is 0, then its results are the same as training
    "holdout_period": 252 * 3,
}

# model params
model_config = {
    "train_steps": train_steps,
    "tasks_train_steps": 0,  # [len(data_config[x]) for x in data_config["region"]] - right
    "batch_size": 32,
    "seq_len": 252,
    "transfer_strat": MODE,
    "device": torch.device(device_config),
    "export_losses": False,
    "no_transfer_linear": {
        "optimizer_learning_rate": 0.001,
        "amsgrad": True,
        "export_weights": False,
    },
    "no_transfer_lstm": {
        "optimizer_learning_rate": 0.001,
        "amsgrad": True,
        "export_model": False,
        "out_n_hi": 50,
        "n_layers": 2,
        "drop_rate": 0.1,
    },
    "global_linear_linear": {
        "optimizer_learning_rate": 0.01,
        "amsgrad": True,
        "export_weights": False,
        "in_transfer_dim": 200,
        "out_transfer_dim": 200,
    },
    "global_lstm_linear": {
        "optimizer_learning_rate": 0.01,
        "amsgrad": True,
        "export_model": False,
        "in_transfer_dim": 5,
        "out_transfer_dim": 5,
        "n_layers": 2,
        "drop_rate": 0.1,
    },
    "global_linear_lstm": {
        "optimizer_learning_rate": 0.01,
        "amsgrad": True,
        "export_model": False,
        "in_transfer_dim": 5,
        "out_transfer_dim": 5,
        "in_n_layers": 2,
        "out_n_layers": 2,
        "out_n_hi": 10,
        "drop_rate": 0.1,
    },
    "global_lstm_lstm": {
        "optimizer_learning_rate": 0.01,
        "amsgrad": True,
        "export_model": False,
        "in_transfer_dim": 5,
        "out_transfer_dim": 5,
        "in_n_layers": 2,
        "out_n_layers": 2,
        "n_layers": 2,
        "out_n_hi": 10,
        "drop_rate": 0.1,
        "drop_rate_transfer": 0.1,
    },
}


## MAIN ROUTINE

In [4]:
# main routine

# pre-allocation
export_label = (
    "valperiod_"
    + str(problem_config["val_period"])
    + "_testperiod_"
    + str(problem_config["holdout_period"])
    + "_tsteps_"
    + str(model_config["train_steps"])
    + "_tksteps_"
    + str(model_config["tasks_train_steps"])
    + "_batchsize_"
    + str(model_config["batch_size"])
    + "_seqlen_"
    + str(model_config["seq_len"])
    + "_transferstrat_"
    + model_config["transfer_strat"]
    + "_lr_"
    + str(model_config[model_config["transfer_strat"]]["optimizer_learning_rate"])
)
data_config["export_label"] = export_label
problem_config["export_label"] = export_label
model_config["export_label"] = export_label
model_config["export_path"] = problem_config["export_path"]


In [5]:
# get data
Xtrain_tasks, Xval_tasks, Xtest_tasks = utils.get_data(
    data_config, problem_config, model_config
)


Americas Americas_IBOV torch.Size([5623, 29])
Americas Americas_MEXBOL torch.Size([5623, 13])
Americas Americas_MERVAL torch.Size([5623, 10])
Americas Americas_SPTSX torch.Size([5623, 108])
Americas Americas_SPX torch.Size([5623, 352])
Americas Americas_RTY torch.Size([5623, 503])
Europe Europe_AEX torch.Size([5623, 17])
Europe Europe_ASE torch.Size([5623, 51])
Europe Europe_ATX torch.Size([5623, 13])
Europe Europe_BEL20 torch.Size([5623, 14])
Europe Europe_BUX torch.Size([5623, 8])
Europe Europe_BVLX torch.Size([5623, 17])
Europe Europe_CAC torch.Size([5623, 35])
Europe Europe_CYSMMAPA torch.Size([5623, 42])
Europe Europe_DAX torch.Size([5623, 25])
Europe Europe_HEX torch.Size([5623, 57])
Europe Europe_IBEX torch.Size([5623, 23])
Europe Europe_ISEQ torch.Size([5623, 10])
Europe Europe_KFX torch.Size([5623, 12])
Europe Europe_OBX torch.Size([5623, 17])
Europe Europe_OMX torch.Size([5623, 29])
Europe Europe_SMI torch.Size([5623, 18])
Europe Europe_UKX torch.Size([5623, 63])
Europe Europ

In [6]:
print(model_config["transfer_strat"])
# set model
if model_config["transfer_strat"] == "no_transfer_linear":
    transfer_trad_strat = no_transfer_linear.NoTransferLinear(
        Xtrain_tasks, model_config
    )
    add_label = [""] * len(data_config["region"])

elif model_config["transfer_strat"] == "no_transfer_lstm":
    transfer_trad_strat = no_transfer_lstm.NoTransferLSTM(Xtrain_tasks, model_config)
    add_label = [
        "_n_hi_"
        + str(model_config["no_transfer_lstm"]["out_n_hi"])
        + "_n_layers_"
        + str(model_config["no_transfer_lstm"]["n_layers"])
        + "dpr"
        + str(model_config["no_transfer_lstm"]["drop_rate"])
        for x in data_config["region"]
    ]

elif model_config["transfer_strat"] == "global_linear_linear":
    transfer_trad_strat = global_linear_linear.GlobalLinearLinear(
        Xtrain_tasks, model_config
    )
    add_label = [
        "_indim_"
        + str(model_config["global_linear_linear"]["in_transfer_dim"])
        + "_outdim_"
        + str(model_config["global_linear_linear"]["out_transfer_dim"])
        for x in data_config["region"]
    ]

elif model_config["transfer_strat"] == "global_lstm_linear":
    transfer_trad_strat = global_lstm_linear.GlobalLSTMLinear(
        Xtrain_tasks, model_config
    )
    add_label = [
        "_indim_"
        + str(model_config["global_lstm_linear"]["in_transfer_dim"])
        + "_outdim_"
        + str(model_config["global_lstm_linear"]["out_transfer_dim"])
        + "_inlay_"
        + str(model_config["global_lstm_linear"]["n_layers"])
        + "dpr"
        + str(model_config["global_lstm_linear"]["drop_rate"])
        for x in data_config["region"]
    ]

elif model_config["transfer_strat"] == "global_linear_lstm":
    transfer_trad_strat = global_linear_lstm.GlobalLinearLSTM(
        Xtrain_tasks, model_config
    )
    add_label = [
        "_indim_"
        + str(model_config["global_linear_lstm"]["in_transfer_dim"])
        + "_outdim_"
        + str(model_config["global_linear_lstm"]["out_transfer_dim"])
        + "_inlay_"
        + str(model_config["global_linear_lstm"]["in_n_layers"])
        + "_outlay_"
        + str(model_config["global_linear_lstm"]["out_n_layers"])
        + "_lindim_"
        + str(model_config["global_linear_lstm"]["out_n_hi"])
        + "dpr"
        + str(model_config["global_linear_lstm"]["drop_rate"])
        for x in data_config["region"]
    ]

elif model_config["transfer_strat"] == "global_lstm_lstm":
    transfer_trad_strat = global_lstm_lstm.GlobalLSTMLSTM(Xtrain_tasks, model_config)
    add_label = [
        "_indim_"
        + str(model_config["global_lstm_lstm"]["in_transfer_dim"])
        + "_outdim_"
        + str(model_config["global_lstm_lstm"]["out_transfer_dim"])
        + "_inlay_"
        + str(model_config["global_lstm_lstm"]["in_n_layers"])
        + "_outlay_"
        + str(model_config["global_lstm_lstm"]["out_n_layers"])
        + "_odim_"
        + str(model_config["global_lstm_lstm"]["out_n_hi"])
        + "_ltr_"
        + str(model_config["global_lstm_lstm"]["n_layers"])
        + "_dpr_"
        + str(model_config["global_lstm_lstm"]["drop_rate"])
        + "_dtr_"
        + str(model_config["global_lstm_lstm"]["drop_rate_transfer"])
        for x in data_config["region"]
    ]


global_lstm_lstm
Americas Americas_IBOV LSTM(29, 5, num_layers=2, batch_first=True, dropout=0.1) LSTM(5, 10, num_layers=2, batch_first=True, dropout=0.1) Linear(in_features=10, out_features=29, bias=True) LSTM(5, 5, num_layers=2, batch_first=True, dropout=0.1) Tanh() Adam (
Parameter Group 0
    amsgrad: True
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)
Americas Americas_MEXBOL LSTM(13, 5, num_layers=2, batch_first=True, dropout=0.1) LSTM(5, 10, num_layers=2, batch_first=True, dropout=0.1) Linear(in_features=10, out_features=13, bias=True) LSTM(5, 5, num_layers=2, batch_first=True, dropout=0.1) Tanh() Adam (
Parameter Group 0
    amsgrad: True
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: F

In [7]:
# additional labelling
to_add_label = {}
for lab, region in zip(add_label, data_config["region"]):
    to_add_label[region] = lab


## TRAIN

In [8]:
# train model
import time

start = time.time()
transfer_trad_strat.train()
print(time.time() - start)

# get signals
Xtrain_signal = transfer_trad_strat.predict(Xtrain_tasks)
Xval_signal = transfer_trad_strat.predict(Xval_tasks)
Xtest_signal = transfer_trad_strat.predict(Xtest_tasks)


16.6329026222229


In [9]:
# compute results
k = True
for region in data_config["region"]:
    region_task_paths = [t + "_all_assets_data.pkl.gz" for t in data_config[region]]

    z = True
    for task, tk_path in zip(data_config[region], region_task_paths):
        # get signal
        pred_train = Xtrain_signal[region][task].cpu()
        pred_val = Xval_signal[region][task].cpu()
        pred_test = Xtest_signal[region][task].cpu()

        # get target
        Ytrain = (
            Xtrain_tasks[region][task]
            .view(1, -1, Xtrain_tasks[region][task].size(1))[:, 1:]
            .cpu()
        )
        Yval = (
            Xval_tasks[region][task]
            .view(1, -1, Xval_tasks[region][task].size(1))[:, 1:]
            .cpu()
        )
        Ytest = (
            Xtest_tasks[region][task]
            .view(1, -1, Xtest_tasks[region][task].size(1))[:, 1:]
            .cpu()
        )

        # compute returns
        df_train_ret = (
            pred_train.mul(Ytrain)[0].cpu().numpy()
            - utils.compute_trading_costs(pred_train)[0].cpu().numpy()
        )
        df_val_ret = (
            pred_val.mul(Yval)[0].cpu().numpy()
            - utils.compute_trading_costs(pred_val)[0].cpu().numpy()
        )
        df_test_ret = (
            pred_test.mul(Ytest)[0].cpu().numpy()
            - utils.compute_trading_costs(pred_test)[0].cpu().numpy()
        )

        # get performance metrics
        df = pd.read_pickle(data_config["data_path"] + tk_path)
        df_train_ret = pd.DataFrame(df_train_ret, columns=df.columns)
        df_train_metrics = utils.compute_performance_metrics(df_train_ret)
        df_train_metrics["exchange"] = task

        df_val_ret = pd.DataFrame(df_val_ret, columns=df.columns)
        df_val_metrics = utils.compute_performance_metrics(df_val_ret)
        df_val_metrics["exchange"] = task

        df_test_ret = pd.DataFrame(df_test_ret, columns=df.columns)
        df_test_metrics = utils.compute_performance_metrics(df_test_ret)
        df_test_metrics["exchange"] = task

        if z:
            all_df_train_metrics = df_train_metrics.copy()
            all_df_val_metrics = df_val_metrics.copy()
            all_df_test_metrics = df_test_metrics.copy()
            z = False
        else:
            all_df_train_metrics = pd.concat(
                [all_df_train_metrics, df_train_metrics], axis=0
            )
            all_df_val_metrics = pd.concat([all_df_val_metrics, df_val_metrics], axis=0)
            all_df_test_metrics = pd.concat(
                [all_df_test_metrics, df_test_metrics], axis=0
            )

    # export results
    all_df_train_metrics["region"] = region
    all_df_train_metrics["set"] = "train"
    all_df_val_metrics["region"] = region
    all_df_val_metrics["set"] = "val"
    all_df_test_metrics["region"] = region
    all_df_test_metrics["set"] = "test"

    pd.concat(
        [all_df_train_metrics, all_df_val_metrics, all_df_test_metrics], axis=0
    ).to_csv(
        problem_config["export_path"]
        + region
        + "_"
        + problem_config["export_label"]
        + to_add_label[region]
        + ".csv"
    )
    pickle.dump(
        model_config,
        open(
            problem_config["export_path"]
            + region
            + "_"
            + problem_config["export_label"]
            + to_add_label[region]
            + "_modelconfig.pkl.gz",
            "wb",
        ),
    )
    if model_config["export_losses"]:
        pickle.dump(
            transfer_trad_strat.losses[region],
            open(
                problem_config["export_path"]
                + region
                + "_"
                + problem_config["export_label"]
                + to_add_label[region]
                + "_losses.pkl.gz",
                "wb",
            ),
        )


d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes\Khoa học dữ liệu\Luận văn tốt nghiệp Thạc sĩ Khoa học dữ liệu\QuantNet\.venv\Lib\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
d:\OneDrive - fpts.com.vn\DucNN privates\Files\Codes

# SAVE AND RELOAD

In [10]:
transfer_trad_strat.save_model(".\\Saved_models\\")

Model saved to: .\Saved_models\global_lstm_lstm_trainsteps_1.pkl.gz


In [14]:
transfer_trad_strat.test = global_lstm_lstm.GlobalLSTMLSTM.load_model(
    ".\\Saved_models\\global_lstm_lstm_trainsteps_1.pkl.gz"
)


In [15]:
transfer_trad_strat.predict(Xtest_tasks)

{'Americas': {'Americas_IBOV': tensor([[[ 0.2520,  0.0264,  0.0715,  ...,  0.0474,  0.2388, -0.2519],
           [ 0.2366,  0.0513,  0.0931,  ...,  0.0764,  0.2279, -0.2307],
           [ 0.2294,  0.0644,  0.1045,  ...,  0.0934,  0.2225, -0.2169],
           ...,
           [ 0.2227,  0.0744,  0.1076,  ...,  0.1008,  0.2419, -0.2114],
           [ 0.2253,  0.0779,  0.1078,  ...,  0.1026,  0.2376, -0.2082],
           [ 0.2240,  0.0778,  0.1118,  ...,  0.0954,  0.2401, -0.2154]]],
         dtype=torch.float64),
  'Americas_MEXBOL': tensor([[[ 0.0457,  0.1778,  0.1918,  ...,  0.0230,  0.2220, -0.2639],
           [ 0.0472,  0.1969,  0.1855,  ...,  0.0253,  0.2519, -0.2557],
           [ 0.0426,  0.2069,  0.1832,  ...,  0.0283,  0.2698, -0.2473],
           ...,
           [ 0.0449,  0.2181,  0.1915,  ...,  0.0567,  0.3010, -0.2487],
           [ 0.0465,  0.2177,  0.1916,  ...,  0.0603,  0.3003, -0.2503],
           [ 0.0404,  0.2195,  0.1906,  ...,  0.0509,  0.3010, -0.2465]]],
         